# LeTour data set 
This file downloads raw data about every rider of every Tour de France (from 1903 up to 2020). This data will then be postprocessed and stored in CSV format.
Executing this notebook might take some minutes.

## 1) Retrieve urls for data extract
First we generate the urls that we need to download the raw HTML pages from the `letour.fr` website to work offline from here on. The dataframe dflink will stores the respective url for each year.
Take a look at `view-source:https://www.letour.fr/en/history` (at around line 1143-1890).  

In [1]:
import os
import subprocess
import collections
import numpy as np
import re
import pandas as pd
import pandas_read_xml as pdx
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import datetime
import time



In [2]:
prefix = 'http://www.letour.fr'
historypage = 'https://www.letour.fr/en/history'


In [3]:
headers = {'Accept': 'text/html', 'User-Agent': 'python-requests/1.2.0','Accept-Charset': 'utf-8','accept-encoding': 'deflate, br'}

In [4]:
resulthistpage = requests.get(historypage, allow_redirects=True, headers=headers)
souphistory = BeautifulSoup(resulthistpage.text, 'html.parser')

In [5]:
# Find select tag for histo links
select_tag_histo = souphistory.find_all("button",{"class" :"dateTabs__link"})

LH = [x["data-tabs-ajax"] for x in select_tag_histo]

dflink = pd.DataFrame({'TDFHistorylink':LH})
dflink

,TDFHistorylink
0,/en/block/history/10707/d0ab6a216569236433268b...
1,/en/block/history/10708/0b76b8f809ad5d8bcf3579...
2,/en/block/history/10709/c5f53ced72a23f333cc186...
3,/en/block/history/10710/ead1d1704b1600c795619e...
4,/en/block/history/10711/f558d3bc819c8ee5dc627d...
...,...
102,/en/block/history/10809/7be3a459d846b4672915c5...
103,/en/block/history/10810/7035a5dc53631209d3581f...
104,/en/block/history/11818/f34c3404d95a697dcf77d4...
105,/en/block/history/11819/96c0eb3fa403ebf222f28b...


## 2) Get data from HTML pages and convert results to dataframes

### 2.1) Create function that convert HHh mm' ss'' to seconds

In [6]:
def calcTotalSeconds(row,mode):

   val = sum(x * int(t) for x, t in zip([3600, 60, 1], row.replace("h",":").replace("'",":").replace('"',':').replace(" ","").replace("+","").replace("-","0").split(":")))
   
   if ((mode=='Gap') and val > 180000) :
        val=0
    
   return val

### 2.2) Create a function that will retrieve elements from a source HTML page located on an input url

In [7]:
def getstagesNrank(i_url):
    resultfull = requests.get(i_url, allow_redirects=True)
    result = resultfull.text
    resultstatus = resultfull.status_code
    
    print(i_url + ' ==> HTTP STATUS = ' + str(resultstatus))
    
    soup = BeautifulSoup(result, 'html.parser')  
    h=soup.find('h3')
    year=int(h.text[-4:])

    # Find select tag
    select_tag = soup.find("select")

    # find all option tag inside select tag
    options = select_tag.find_all("option")

    cols = ['Year','TotalTDFDistance','Stage']
    lst = []
    
    #search for stages
    distance=soup.select("[class~=statsInfos__number]")[1].contents
               
    #search for distance of the TDF edition
    for option in options:
       lst.append([year,int(distance[0].replace(" ","")),option.text])
               
    dfstages = pd.DataFrame(lst, columns=cols)
    
    
    # Find select tag for ranking racers
    rankingTable = soup.find("table")

    dfrank = pd.read_html(str(rankingTable))[0]

    dfrank['Year']=year
    dfrank['Distance (km)']=int(distance[0].replace(" ",""))
    dfrank['Number of stages']=len(dfstages)
    dfrank['TotalSeconds']=dfrank['Times'].apply(lambda x:calcTotalSeconds(x,'Total'))
    dfrank['GapSeconds']=dfrank['Gap'].apply(lambda x:calcTotalSeconds(x,'Gap'))

    
    return dfstages, dfrank

### 2.3) Loop on the dflink dataframe to get data from each url source

In [15]:
dfstagesres=[]
dfstagesrestmp=[]
dfrankres=[]
dfrankrestmp=[]
dfrankoutput=[]

for index, row  in dflink.iterrows():
    url = prefix+row['TDFHistorylink']
    try :
      if index >= 12 :  # limit from 1919 (data need to be cleaned a little bit more before that)
        dfstagesres, dfrankres=getstagesNrank(url)
        dfstagesrestmp= dfstagesres.append(dfstagesrestmp, ignore_index=True)  
        dfrankrestmp=dfrankres.append(dfrankrestmp, ignore_index=True)
        
        
    except:
      raise

    
dfstageoutput=dfstagesrestmp
dfrankoutput=dfrankrestmp

http://www.letour.fr/en/block/history/10719/f15cf81b7599ea113d2b1d614f73bf83 ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10720/53ac40095834ae1aae5d197aa730799a ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10721/c663ba39ac60d58c55a1a787a23025fc ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10722/cc5d0ba48a5d9bd9ee5e222160452b6f ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10723/9212137527c3f19c0cfccd1d07d80649 ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10724/61621b1b7012e711ae7457a04e123aab ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10725/326ccc005b547ad2ad33e2fc03d27159 ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10726/417b583aab876fdb00496da5c2be1a7d ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10727/0ab2dd4f425fcdcb9cc0a1511f49c494 ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10728/58abb1c5f80d5d1b088ead3c32a5a815 ==> HTTP STATUS = 200
http://www

http://www.letour.fr/en/block/history/10802/814bff61212e58cf7c2beaddb1c083b3 ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10803/757e8e81b7183fde7329fd8ab9b69783 ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10804/89b36b01ffe439e016ec1d59c57b63d1 ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10805/1ec24b4375d16b922c9c9c489d25465c ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10806/196e36cfe7aff4b5fc2d1055f7dcf864 ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10807/d538a7fbdfc0d657fbf064561840fbb1 ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10808/3f166bedb535ee9bff2fb7ead0a7812c ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10809/7be3a459d846b4672915c576cb7ed6b9 ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10810/7035a5dc53631209d3581f64a433b10d ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/11818/f34c3404d95a697dcf77d4cd8e8278fa ==> HTTP STATUS = 200
http://www

In [9]:
dfrankoutput

,Rank,Rider,Rider No.,Team,Times,Gap,B,P,Year,Distance (km),Number of stages,TotalSeconds,GapSeconds
0,1,TADEJ POGAČAR,131,UAE TEAM EMIRATES,87h 20' 05'',-,32',NaN,2020,3483,21,314405,0
1,2,PRIMOŽ ROGLIČ,11,TEAM JUMBO - VISMA,87h 21' 04'',+ 00h 00' 59'',33',NaN,2020,3483,21,314464,59
2,3,RICHIE PORTE,101,TREK - SEGAFREDO,87h 23' 35'',+ 00h 03' 30'',04',NaN,2020,3483,21,314615,210
3,4,MIKEL LANDA MEANA,61,BAHRAIN - MCLAREN,87h 26' 03'',+ 00h 05' 58'',NaN,NaN,2020,3483,21,314763,358
4,5,ENRIC MAS NICOLAU,94,MOVISTAR TEAM,87h 26' 12'',+ 00h 06' 07'',NaN,NaN,2020,3483,21,314772,367
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9062,6,JACQUES COOMANS,63,TDF 1919 ***,246h 28' 49'',+ 15h 21' 34'',NaN,NaN,1919,5560,15,887329,55294
9063,7,LUIGI LUCOTTI,19,TDF 1919 ***,247h 08' 27'',+ 16h 01' 12'',NaN,NaN,1919,5560,15,889707,57672
9064,8,JOSEPH VAN DAELE,55,TDF 1919 ***,249h 30' 17'',+ 18h 23' 02'',NaN,NaN,1919,5560,15,898217,66182
9065,9,ALFRED STEUX,56,TDF 1919 ***,251h 36' 16'',+ 20h 29' 01'',NaN,NaN,1919,5560,15,905776,73741


## 3) Clean up the data 


In [10]:
# Fix result types
dfrankoutput["ResultType"] = "time"
dfrankoutput.loc[dfrankoutput["Year"].isin([1905,1906,1908]),"ResultType"] = "null"
dfrankoutput.loc[dfrankoutput["Year"].isin([1907,1909,1910,1911,1912]),"ResultType"] = "points"

In [11]:
# Fix this weird bug for e.g. year 2006 and 1997
for year in np.unique(dfrankoutput["Year"]):
    tmp = dfrankoutput[dfrankoutput["Year"]==year].reset_index()
    if tmp.loc[0]["TotalSeconds"] > tmp.loc[2]["TotalSeconds"]:
        print(year)
# Okay seems to be only for 2006 and 1997


1997
2006


In [12]:
tmp = dfrankoutput[dfrankoutput["Year"]==2006].reset_index()
ts = np.array(tmp["TotalSeconds"])
gs = np.array(tmp["GapSeconds"])
ts[1:] = ts[0]+gs[1:]

dfrankoutput.loc[dfrankoutput["Year"]==2006,"TotalSeconds"] = ts

tmp = dfrankoutput[dfrankoutput["Year"]==1997].reset_index()
ts = np.array(tmp["TotalSeconds"])
gs = np.array(tmp["GapSeconds"])
ts[1:] = ts[0]+gs[1:]

dfrankoutput.loc[dfrankoutput["Year"]==1997,"TotalSeconds"] = ts

## 4) Write Data to CSV

In [13]:
dfrankoutput.to_csv("../data/TDF_Riders_History.csv")

In [14]:
dfstageoutput.to_csv("../data/TDF_Stages_History.csv")